In [1]:
#export

# temptation
# night watcher to avoid unexpected halt 
# when performing request, which freeze the program and won't be able to throw exception
# the best way to limit the run time of a fn, is to run as subprocess
# subprocessing and manipuating threads is too raw for me
# so, i use async program model to deal with the mess

# the program still freeze sometimes
# https://stackoverflow.com/questions/49572547/python-application-freezes-only-ctrl-c-helps
# https://stackoverflow.com/questions/29649173/what-is-the-global-default-timeout
import socket
socket.setdefaulttimeout(30.)

import os, sys
from os.path import join
wd = os.getcwd()
pyf = join(wd, 'nbexp_main.py')
code = f'{sys.executable} {pyf}'

In [2]:
#export
from functools import partial
print=partial(print, flush=True)
in_jupyter = (__doc__ == 'Automatically created module for IPython interactive environment')

In [3]:
#export

# https://docs.python.org/3/library/asyncio-subprocess.html
import asyncio
import sys
import datetime
import time
# https://stackoverflow.com/questions/44633458/why-am-i-getting-notimplementederror-with-async-and-await-on-windows
asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy()) 


async def get_date(run_args=[sys.executable, '-c', "import time;print('ing',  flush=True);time.sleep(2);print('done')",]
                   , wait_child_sec=3):

    # Create the subprocess; redirect the standard output
    # into a pipe.
    proc = await asyncio.create_subprocess_exec(
        *run_args,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE,
    )

    await asyncio.wait_for(proc.wait(), timeout=wait_child_sec)
    
    line=''
    
    data = await proc.stdout.read()
    line += data.decode('ascii').rstrip()
    
    line += '\n'
    
    data = await proc.stderr.read()
    line += data.decode('ascii').rstrip()
    
    if line: print(line)
    

In [4]:
# testing
if not in_jupyter: asyncio.run(get_date())

In [5]:
#export
hrs = 0.5
sec = hrs * 60 * 60
from time import sleep
import traceback
from nbexp_main import timer, save_notebook

while True:
    if in_jupyter:
        break;
    try:
        t = timer()
        next(t)
        date = asyncio.run(get_date([sys.executable, pyf,], 30))
        next(t)
    except asyncio.TimeoutError:
        print('timeout!')
    except Exception as e:
        print("unexpected err")
        print(traceback.print_exc())
        
    # better behaviour if your put computer to sleep
    t = timer(slient_mode=True)
    while True:
        sleep(10)
        _, time_cost = next(t)
        if time_cost > sec:
            break;

In [6]:
save_notebook()

<IPython.core.display.Javascript object>

In [ ]:
!python notebook2script.py watcher.ipynb
save_notebook()

Converted watcher.ipynb to nbexp_watcher.py


In [27]:
# !{code}

In [ ]:
# !python nbexp_watcher.py